# How to access open Earth observation training data
### **Assessment Two**

## Example application of learning a crop type classification model using training data from Radiant ML Hub
---
This Python notebook takes you through an application of learning  a classification model for classifying crop types against field IDs in a satellite image. 

---
The notebook is based on a starter notebook provided for the 2020 ICLR workshop challenge at Zindi on using earth observation data for classifying crop types during the growing season: https://zindi.africa/competitions/iclr-workshop-challenge-2-radiant-earth-computer-vision-for-crop-recognition

---
This is a high level overview of the notebook:

- Download data (tif files and label and field information) from Radiant ML Hub
- Do data preprocessing by sampling the satellite images to get the data into an easy tabular form 
- Learn a classification model on this data, and experiment with grouping by field or doing pixel-wise classifications.


## **Download Data from Radiant ML Hub**

At the end of this section you should have downloaded all images and labels of the crop type dataset that constitutes satellite imagery over western Kenya labeled with field IDs and respective crop types.

The code in this section is essentially a clone from a starter notebook provided by Radiant Earth for downloading satellite images labeled with field IDs and crop types: https://github.com/radiantearth/mlhub-tutorials/tree/master/notebooks/2020%20CV4A%20Crop%20Type%20Challenge

In this section we will:

- import libraries that we use to access and get information from the Radiant ML Hub API
- specify authentication details and API properties that we shall use whenever we access the API
- specify the download location where the dataset will be downloaded
- define functions to use for donwloading labels and images
- specify collection IDs and download the respective labels and images.



### **Import required libraries**

In [ ]:
# Required libraries
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime

### **Authentication**

Access to the Radiant MLHub API requires an access token. To get your access token, go to dashboard.mlhub.earth. If you have not used Radiant MLHub before, you will need to sign up and create a new account. Otherwise, sign in. Under Usage, you'll see your access token, which you will need. Do not share your access token with others: your usage may be limited and sharing your access token is a security risk.

You can copy and paste your access token in the value for the ACCESS_TOKEN variable in this next cell. Then headers block will be used in all our requests to Radiant ML Hub's API.



In [ ]:
ACCESS_TOKEN = 'PASTE THE TOKEN FROM YOUR RADIANT MLHUB DASHBOARD HERE'
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}',
    'Accept':'application/json'
}

#### **Specify the output path to the folder where the data will be downloaded**

You can first get the dataset to Google colab and use it there, or copy it to your data folder in Google drive, or download it to your data folder on your local machine.

In this case, we mount Google Drive and specify the path to the data folder where the images and labels will be downloaded.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
output_path = Path("/content/drive/My Drive/Colab Notebooks/data/")

### **Function definitions for downloading the labels and source imagery**

In [ ]:
def get_download_url(item, asset_key, headers):
    asset = item.get('assets', {}).get(asset_key, None)
    if asset is None:
        print(f'Asset "{asset_key}" does not exist in this item')
        return None
    r = requests.get(asset.get('href'), headers=headers, allow_redirects=False)
    return r.headers.get('Location')

def download_label(url, output_path, tileid):
    filename = urlparse(url).path.split('/')[-1]
    outpath = output_path/tileid
    outpath.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    f = open(outpath/filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk:
            f.write(chunk)
    f.close()
    print(f'Downloaded {filename}')
    return 

def download_imagery(url, output_path, tileid, date):
    filename = urlparse(url).path.split('/')[-1]
    outpath = output_path/tileid/date
    outpath.mkdir(parents=True, exist_ok=True)
    
    r = requests.get(url)
    f = open(outpath/filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk:
            f.write(chunk)
    f.close()
    print(f'Downloaded {filename}')
    return

### **Retrieving the data**

Datasets are stored as collections on Radiant MLHub catalog. A collection represents the top-most data level. Typically this means the data comes from the same source for the same geography. It might include different years or sub-geographies.

The two collections we shall use in the machine learning application are:

- ref_african_crops_kenya_02_source: includes the multi-temporal bands of Sentinel-2
- ref_african_crops_kenya_02_labels: includes the labels and field IDs

#### **Download labels**

The assets property of the items in a collection contains all the assets associated with that item and links to download them. The labels for the item will always be the asset with the key labels. The following code will go through every item in the collection and download the labels and field_ids raster feature.

In [ ]:
# paste the id of the labels collection:
collectionId = 'ref_african_crops_kenya_02_labels'

# these optional parameters can be used to control what items are returned. 
# Here, we want to download all the items so:
limit = 100
bounding_box = []
date_time = []

# retrieves the items and their metadata in the collection
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items', params={'limit':limit, 'bbox':bounding_box,'datetime':date_time}, headers=headers)
collection = r.json()

In [ ]:
# retrieve list of features (in this case tiles) in the collection
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    print("Feature", feature.get('id'), 'with the following assets', list(assets))

In [ ]:
for feature in collection.get('features', []):
    
    tileid = feature.get('id').split('tile_')[-1][:2]

    # download labels
    download_url = get_download_url(feature, 'labels', headers)
    download_label(download_url, output_path, tileid)
    
    #download field_ids
    download_url = get_download_url(feature, 'field_ids', headers)
    download_label(download_url, output_path, tileid)

#### **Download images**

The imagery items associated with the tiles are linked within the links array of the tile metadata. Links which have a rel type of "source" are links to imagery items. By requesting the metadata for the imagery item you can retrieve download URLs for each band of the imagery.

In [ ]:
# paste the id of the imagery collection:
collectionId = 'ref_african_crops_kenya_02_source'

# these optional parameters can be used to control what items are returned. 
# Here, we want to download all the items so:
limit = 100
bounding_box = []
date_time = []

# retrieves the items and their metadata in the collection
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items', params={'limit':limit, 'bbox':bounding_box,'datetime':date_time}, headers=headers)
collection = r.json()

In [ ]:
# List assets of the items
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    print(list(assets))
    break #all the features have the same type of assets. for simplicity we break the loop here.

In [ ]:
# This cell downloads all the multi-spectral images throughout the growing season for this competition.
# The size of data is about 1.5 GB, and download time depends on your internet connection. 
# Note that you only need to run this cell and download the data once.
i = 0
for feature in collection.get('features', []):
    assets = feature.get('assets').keys()
    tileid = feature.get('id').split('tile_')[-1][:2]
    date = datetime.strftime(datetime.strptime(feature.get('properties')['datetime'], "%Y-%m-%dT%H:%M:%SZ"), "%Y%m%d")
    for asset in assets:
        i += 1
        if i > 0: # if resuming after it failed
          download_url = get_download_url(feature, asset, headers)
          download_imagery(download_url, output_path, tileid, date)

# **Viewing the images**

In this section, we will explore the downloaded data set by viewing a single image corresponding one spectral band, then we will view an RGB image from a combination of images corresponding to the RGB colors. 

Again, this section of the notebook contains code from the starter notebooks on loading and visualizing a sample of .tif images from Radiant ML Hub. 

In the code, we use the tifffile package to read .tif images. We use the matplotlib library to view the images.

In [ ]:
# Install the tifffile package which we shall use to read the .tif images
!pip install tifffile

In [ ]:
# Import required libraries for opening and viewing
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# This is a function that we shall use whenever we want to open a .tif image
def load_file(fp):
    """Takes a PosixPath object or string filepath
    and returns np array"""
    
    return tiff.imread(fp.__str__())

#### **Load and view a single .tif image**

In [ ]:
# Sample file to load:
file_name = "/content/drive/My Drive/Colab Notebooks/data/00/20191103/0_B01_20191103.tif"
band_data = load_file(file_name)

In [ ]:
# View the sample file
fig = plt.figure(figsize=(10, 10))
plt.imshow(band_data, vmin=0, vmax=0.15)

#### **View an RGB image**

In this section, we combine a sample of the Red Green and Blue band images to view an RGB image.


In [ ]:
# Quick way to see an RGB image. Can mess with the scaling factor to change brightness (9 in this example)

import numpy as np
def load_rgb(tile, date):

  r = load_file(f"/content/drive/My Drive/Colab Notebooks/data/{tile}/{date}/{tile[1]}_B04_{date}.tif")
  g = load_file(f"/content/drive/My Drive/Colab Notebooks/data/{tile}/{date}/{tile[1]}_B03_{date}.tif")
  b = load_file(f"/content/drive/My Drive/Colab Notebooks/data/{tile}/{date}/{tile[1]}_B02_{date}.tif")
  arr = np.dstack((r, g, b))
  print(max(g.flatten()))
  return arr

fig, ax = plt.subplots(figsize=(12, 18))
ax.imshow(load_rgb('00', '20191103')*9)
plt.tight_layout()

# **Data Pre-processing for machine learning**

Now that you have downloaded and explored the dataset by viewing a sample of images, let's now transform the downloaded data for training and evaluating classificatnon models. 

First of all, we need to get all the dates constituting the observations in the dataset and the bands in each observation into lists that we shall use later to specify images or go through all images.



In [ ]:
# List of dates that an observation from Sentinel-2 is provided in the training dataset

dates = [datetime.datetime(2019, 6, 6, 8, 10, 7),
         datetime.datetime(2019, 7, 1, 8, 10, 4),
         datetime.datetime(2019, 7, 6, 8, 10, 8),
         datetime.datetime(2019, 7, 11, 8, 10, 4),
         datetime.datetime(2019, 7, 21, 8, 10, 4),
         datetime.datetime(2019, 8, 5, 8, 10, 7),
         datetime.datetime(2019, 8, 15, 8, 10, 6),
         datetime.datetime(2019, 8, 25, 8, 10, 4),
         datetime.datetime(2019, 9, 9, 8, 9, 58),
         datetime.datetime(2019, 9, 19, 8, 9, 59),
         datetime.datetime(2019, 9, 24, 8, 9, 59),
         datetime.datetime(2019, 10, 4, 8, 10),
         datetime.datetime(2019, 11, 3, 8, 10)]

In [ ]:
# List of bands in each observation

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

### **Getting data for each pixel in fields**

This is where we start massaging the data into a format we can use. 

We start by reading in the labels, finding the pixel locations of all the fields (most are only a few pixels in size) and storing the pixel locations, the field ID and the label.

In [ ]:
# Not super efficient but  ¯\_(ツ)_/¯
import pandas as pd

row_locs = []
col_locs = []
field_ids = []
labels = []
tiles = []
for tile in range(4):
  fids = f'/content/drive/My Drive/Colab Notebooks/data/0{tile}/{tile}_field_id.tif'
  labs = f'/content/drive/My Drive/Colab Notebooks/data/0{tile}/{tile}_label.tif'
  fid_arr = load_file(fids)
  lab_arr = load_file(labs)
  for row in range(len(fid_arr)):
    for col in range(len(fid_arr[0])):
      if fid_arr[row][col] != 0:
        row_locs.append(row)
        col_locs.append(col)
        field_ids.append(fid_arr[row][col])
        labels.append(lab_arr[row][col])
        tiles.append(tile)

df = pd.DataFrame({
    'fid':field_ids,
    'label':labels,
    'row_loc': row_locs,
    'col_loc':col_locs,
    'tile':tiles
})

print(df.shape)
print(df.groupby('fid').count().shape)
df.head()

In [ ]:
df.head()


---

Then we loop through all the images, sampling the different image bands to get the values for each pixel in each field.

---


In [ ]:
# Sample the bands at different dates as columns in our new dataframe
col_names = []
col_values = []

for tile in range(4): # 1) For each tile
  print('Tile: ', tile)
  for d in dates: # 2) For each date
    print(str(d))
    d = ''.join(str(d.date()).split('-')) # Nice date string
    t = '0'+str(tile)
    for b in bands: # 3) For each band
      col_name = d+'_'+b
      
      if tile == 0:
        # If the column doesn't exist, create it and populate with 0s
        df[col_name] = 0

      # Load im
      im = load_file(f"/content/drive/My Drive/Colab Notebooks/data/{t}/{d}/{t[1]}_{b}_{d}.tif")
      
      # Going four levels deep. Each second on the outside is four weeks in this loop
      # If we die here, there's no waking up.....
      vals = []
      for row, col in df.loc[df.tile == tile][['row_loc', 'col_loc']].values: # 4) For each location of a pixel in a field
        vals.append(im[row][col])
      df.loc[df.tile == tile, col_name] = vals
df.head()


In [ ]:
df.to_csv('sampled_data.csv', index=False)
df.head()

---

We can copy the prepared data (sampled_data.csv) to the data folder in Google Drive

---

In [ ]:
!cp 'sampled_data.csv' '/content/drive/My Drive/Colab Notebooks/data/sampled_data.csv' # You can mount drive and save like this, or just download sampled_data.csv and use that later.

# **Using the prepared data for machine learning crop type classification models**

---

Now that we have our data in a nice tabular data format, we can try some simple machine learning methods using the sklearn package.

We can restart the runtime here and load in the data we saved earlier.

---

### **Import classifiers from sklearn**


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # Random Forest classifier, AdaBoost classifier
from sklearn.neighbors import KNeighborsClassifier # K nearest neighbor classifier
from sklearn.tree import DecisionTreeClassifier # Decision tree classifier
from sklearn.gaussian_process import GaussianProcessClassifier # Gaussian process classifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import log_loss # evaluation metric

### **Load data set**

We load the data set and split it into a training set (for training a classifier) and a test set (for evaluating the classifier).


In [ ]:
# Load the data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/sampled_data.csv')
print(df.shape)
df.head()

In [ ]:
# Split into train and test sets
train = df.loc[df.label != 0]
test =  df.loc[df.label == 0]
train.shape, test.shape

In [ ]:
# Split train into train and validation (val) set, so that we can score our models locally.

# We split on field ID since we want to predict for unseen FIELDS
val_field_ids = train.groupby('fid').mean().reset_index()['fid'].sample(frac=0.3).values
tr = train.loc[~train.fid.isin(val_field_ids)].copy()
val = train.loc[train.fid.isin(val_field_ids)].copy()


### **Classification model training**

We can train a classifier to predict on pixels and then combine, or we can train a classfier by first grouping into fields. Let's do both and compare!

**A. Training and predicting on pixels**

In [ ]:
# Split into X and Y for modelling
X_train, y_train = tr[tr.columns[5:]], tr['label']
X_val, y_val = val[val.columns[5:]], val['label']

In [ ]:
# Train a classification model (takes a few minutes since we have plenty of rows)
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

In [ ]:
# Predict on pixels

# We use the predict_proba() function to get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val dataframe as columns
for i in range(7):
  val[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val['true'+str(c)] = (val['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]

# Calculate the evaluation score - using the log-loss metric
#print('Pixel score:', log_loss(val[true_cols], val[pred_cols]))
print('Field score:', log_loss(val.groupby('fid').mean()[true_cols], val.groupby('fid').mean()[pred_cols])) # This is what we'll use to compare

# Inspect or compare visually
val[['label']+true_cols+pred_cols].head()

**B. Training and predicting on fields**

In [ ]:
# Group training data into fields

train_grouped = tr.groupby('fid').mean().reset_index()
val_grouped = val.groupby('fid').mean().reset_index()
X_train, y_train = train_grouped[train_grouped.columns[5:]], train_grouped['label']
X_val, y_val = val_grouped[train_grouped.columns[5:]], val_grouped['label']

In [ ]:
# Train a classification model
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

In [ ]:
# Get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val_grouped dataframe as columns
for i in range(7):
  val_grouped[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val_grouped['true'+str(c)] = (val_grouped['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]
val_grouped[['label']+true_cols+pred_cols].head()

# Already grouped, but just to double check:
print('Field score:', log_loss(val_grouped.groupby('fid').mean()[true_cols], val_grouped.groupby('fid').mean()[pred_cols]))

## **PRACTICAL EXERCISE**

### **If you were able to complete train and predict in A and B, answer the following questions**

#### **Qn 1.** Which of the two approaches is much faster?

#### **Qn 2.** Which of the two approaches is more accurate?

#### **Qn 3 - Hands on.** Instead of the Random Forest classifier, apply other classifiers from the sklearn package and compare their performance using the log loss metric.